# Synthetic Log Generation from DECLARE Models

DECLARE4Py implements the generation of synthetic logs from DECLARE models with a solution based on Answer Set Programming that uses a Clingo solver. More details can be found in the paper of Chiariello, F., Maggi, F. M., & Patrizi, F. (2022, June). ASP-Based Declarative Process Mining. In *Proceedings of the AAAI Conference on Artificial Intelligence* (Vol. 36, No. 5, pp. 5539-5547).

As first step, it is necessary to import a `.decl` file containing the DECLARE constraints.

In [4]:
import os
from Declare4Py.ProcessModels.Declare4PyModel import DeclareModel

model_name = 'sepsis'
model: DeclareModel = DeclareModel().parse_from_file(os.path.join("../../../", "tests", "test_models", f"{model_name}.decl"))

Then, some general settings are needed to set the number of cases to generate and the minimum and maximum number of events for each case


In [3]:
# Number of cases that have be generated
num_of_cases = 10

# Minimum and maximum number of events a case can contain
(num_min_events, num_max_events) = (3, 10)

The class `AspGenerator` has to be instantiated with the DECLARE model and the settings of above. Then, the `run` method will generate the cases and the `to_xes` method will save them in a `.xes` event log.

In [ ]:
from Declare4Py.ProcessMiningTasks.LogGenerator.ASP.ASPGenerator import AspGenerator

asp_gen: AspGenerator = AspGenerator(model, num_of_cases, num_min_events, num_max_events)
asp_gen.run()